# Midterm

Please hand in the midterm via MarkUs by 9:00am on Thursday, October 26th. You are welcome to spend any amount of time working on the midterm before then. The midterm is "open-course-materials": you are allowed to consult the textbook (d2l.ai), my lecture notes, class recordings, your own course notes, past homeworks, and tutorial materials. You are welcome to check your answers by writing and running code, but this should not be necessary for any of the problems. No collaboration with other students or use of other sources is allowed.

# Problem 1 (2 points)

In class, we introduced the weight decay regularizer, which adds a loss term $\lambda ||w||_2^2$ where $w$ is a vector comprising all of the parameters of the model and $\lambda$ is a scalar hyperparameter. An alternative is L1 regularization, which adds the loss term $\lambda |w|$. Let the model's original, unregularized loss be $L$, so that the regularized loss is $L + \lambda |w|$.
1. What is the gradient of the L1 regularization term with respect to the parameters $w$? For simplicity, you can assume the model has only one parameter, so that $w$ is a scalar.
1. What is the parameter update under normal gradient descent for $L + \lambda |w|$? Since you don't know $L$, you can include a $\nabla_w L$ term in your answer.

1.
$$
\frac{d \lambda|w|}{dw} = \lambda \text{sign}(w) = \lambda (2\mathbb{1}(w > 0) - 1)
$$
2.
\begin{align}
w
& \leftarrow w - \eta(\nabla_w(L + \lambda|w|)) \\
& \leftarrow w - \eta(\nabla_w L + \nabla_w \lambda|w|) \\
& \leftarrow w - \eta(\nabla_w L + \lambda\text{sign}(w))
\end{align}
where $\eta$ is the learning rate.

# Problem 2 (6 points)

Consider the following model and loss function:
\begin{align}
h &= \text{ReLU}(W_hx) \\
o &= W_oh + x \\
L(y, o) &= (y - o)^2 \\
\end{align}
where $x$ is the input, $y \in \mathbb{R}$ is the scalar target, $W_h$ and $W_o$ are learnable parameters, and $o \in \mathbb{R}$ is the scalar output of the model.

1. Derive the expressions for the gradient of the loss with respect to every parameter in the model (that is, $\frac{\partial L}{\partial W_o}$ and $\frac{\partial L}{\partial W_h}$).
1. Re-express the model as a computational graph. The graph can have the following operations: Dot product, addition (not subtraction), elementwise maximum (ReLU), elementwise multiplication, and elementwise power (e.g. squaring). Your graph node should have leaf nodes corresponding to parameters and inputs.
1. Specify which nodes in your graph have an output that needs to be cached in order to use the backpropagaion algorithm.

1.
\begin{align*}
\frac{\partial L}{\partial o} &= -2(y - o) \\
\frac{\partial L}{\partial W_o} &= \frac{\partial L}{\partial o}\frac{\partial o}{\partial W_o} \\
&= -2(y - o)h \\
\frac{\partial o}{\partial W_h} &=  \frac{\partial L}{\partial o}\frac{\partial o}{\partial h}\frac{\partial h}{\partial W_h} \\
&= -2(y - o)W_ox\text{step}(W_hx)
\end{align*}
2.
```
        x
Wh     /|
  \   / |
  dot ⃰  |  # could also use elementwise multiplication since x is a scalar
   |  0 |
   | /  |
 max ⃰   |  # could also use ReLU here
 |  Wo  |
 | /   /
 dot  /
  |  /
  add
   |  -1
   | /
multiply
   |  y
   | /
   add ⃰
    |  2
    | /
    pow  # could also use "square" here
```

3. Nodes whose output needs to be cached are marked above by a  ⃰ (corresponding to $W_h x$, $h$, and $y - o$).

# Problem 3 (3 points)

Consider a convolutional neural network with the following structure:
- Input image of size 32x32x3 (32 width, 32 height, 3 channels)
- Convolutional layer with a 5x5 filter, 32 channels, stride of 1x1, and padding of 2x2
- ReLU nonlinearity
- Batch normalization
- Max pooling with a 2x2 window, stride of 2x2, and no padding
- Convolutional layer with a 3x3 filter, 64 channels, stride of 1x1, and padding of 1x1
- ReLU nonlinearity
- Batch normalization
- Max pooling with a 2x2 window, stride of 2x2, and no padding
- Convolutional layer with a 1x1 filter, 100 channels, stride of 1x1, and no padding

1. What is the output shape (dimensionality/size of the activations) at the output of these layers?
1. What is the receptive field of the input for one of the activations at the output of the model? Recall that the receptive field means the region of the input that influences a given activation.
1. Assume that we remove all padding from all convolutional layers in the model. What is the output shape of these layers now?
1. Now, instead assume that we remove the batch normalization and change the nonlinearities to be sigmoid instead of ReLU. What is the output shape of these layers now?
1. Say we want to use this model for 10-class classification, so we add a single classification layer (i.e. a softmax regression model) to the output. How many parameters will this classification layer have?
1. Imagine instead that we add a global average pooliong layer on the output of the layer stack before the 10-way softmax classification layer. How many parameters will the classification layer have now?

1.
* input: (32, 32, 3)
* after first conv: (32, 32, 32)
* after relu: (32, 32, 32)
* after batchnorm: (32, 32, 32)
* after max pool: (16, 16, 32)
* after conv: (16, 16, 64)
* after relu: (16, 16, 64)
* after batchnorm: (16, 16, 64)
* after max pool: (8, 8, 64)
* after conv: (8, 8, 100)

2. 12x12

3.

* input: (32, 32, 3)
* after first conv: (28, 28, 32)
* after relu: (28, 28, 32)
* after batchnorm: (28, 28, 32)
* after max pool: (14, 14, 32)
* after conv: (12, 12, 64)
* after relu: (12, 12, 64)
* after batchnorm: (12, 12, 64)
* after max pool: (6, 6, 64)
* after conv: (6, 6, 100)

4. Batch norm and the choice of (elementwise) nonlinearity do not change the shape, so removing batch norm and changing the nonlinearity will not change the shape (i.e., same answer as in part 1).

5. Since the output shape is (8, 8, 100) = 6,400-dimensional and the classification layer needs to output 10 scores, the classification weight matrix will need to be 6,400 x 10 and the weight matrix will have 10 values, so 64,010 parameters total.

6. Global average pooling will reduce over the height and width dimensions, producing a 100-dimensional vector. The weight matrix will then have 1,000 parameters, so 1,010 parameters total.

# Problem 4 (2 points)

1. In the olden days of neural networks, it was popular to use the following nonlinearity: $$\phi(x) = \begin{cases} 0, x < 0\\1, x \ge 0\end{cases}$$ Can you foresee any issue with using gradient descent to train a neural network that uses this nonlinearity?
1. An alternative to the ReLU nonlinearity is the softplus: $$\phi(x) = \log(1 + \exp(x))$$ What is the gradient of this nonlinearity with respect to x? Name one advantage and one disadvantage of this nonlinearity over ReLU.

1. This function has a gradient of 0 almost everywhere, so if you use gradient descent to train such a model, any parameters that appear after the use of this nonlinearity will not be updated. Also, it's discontinuous at 0, so the gradient is not defined everywhere in the function's domain.

2. $\frac{\partial \phi}{\partial x} = \frac{\exp(x)}{1 + \exp(x)} = \frac{1}{1 + \exp(-x)} = \sigma(x)$
This function has a continuous nonzero derivative and is nonzero over its full domain. However, it would be more expensive to compute thanks to the log and exp. You'd also have to be careful about overflow.

# Problem 5 (4 points)

A less-popular alternative to batch and layer normalization is weight normalization, which replaces each weight vector $w$ in the model with $\frac{\gamma}{||v||}v$ where $\gamma$ is a new scalar parameter and $v$ is a new vector of parameters with the same shape as $w$.

1. Derive expressions for $\nabla_\gamma L$ and $\nabla_v L$ in terms of $\gamma$, $v$, and $\nabla_w L$.

2. Show that applying batch normalization *without the shift parameter $\beta$* to the preactivation $vx$ is equivalent to applying weight normalization if the entries of $x$ are independently distributed with zero mean and unit variance.


1. $\nabla_\gamma L = \nabla_wL \frac{\partial w}{\partial \gamma} = \nabla_w L \frac{v}{||v||}$

\begin{align*}
\nabla_v L &= \nabla_wL \frac{\partial w}{\partial v}\\
&= \nabla_wL\left(\frac{\gamma}{||v||} \frac{\partial v}{\partial v} + v\frac{\partial (\gamma/||v||)}{\partial v}\right) \\
&= \nabla_wL \left(\frac{\gamma}{||v||} + v \left(\frac{-\gamma}{||v||^2}\right) \frac{v}{||v||}\right)
\end{align*}

2. For the preactivation $vx$, $\text{BN}(vx) = \gamma \frac{vx - \hat{\mu}_B}{\hat{\sigma}_B + \epsilon} + \beta$.
Ignoring $\beta$ and assume $\epsilon$ is negligibly small. $\hat{\mu}_B$ will be 0 (in expectation) since $x$ is zero-mean and therefore $vx$ is zero-mean. If $x$ is unit-variance, then the standard deviation of $vx$ is $||v||$. So $\text{BN}(vx) = \gamma \frac{vx}{||v||}$ which is equivalent to applying weight normalization to $w$.

# Problem 6 (3 points)

Consider the Adam optimizer and assume that $g_t = g \ne 0$, i.e. the gradient is the same value $g \ne 0$ at all iterations of training.

1. Assume that $\epsilon = 0$, $0 < \beta < 1$, and $0 < \beta_2 < 1$ and show that $g^\prime_t = \eta\, \text{sign}(g)$.
2. Now, consider the case when $\epsilon$ is large, e.g. $\epsilon = 1$. How does the function for $g^\prime_t$ change?

1. Following the reasoning in class, when $g_t = g$ we have $v_t = \beta_1^t v_0 + (1 - \beta_1^t) g = (1 - \beta_1^t) g $ since $v_0 = 0$. Then $\hat{v}_t = g$. Similarly we can see that $\hat{s}_t = g^2$. Then $g'_t = \frac{\eta \hat{v}_t}{\sqrt{\hat{s}_t} + \epsilon} = \eta g\big/\sqrt{g^2} = \eta \text{sign}(g)$.
2. If $\epsilon = 1$, then $g'_t = \frac{\eta g}{\sqrt{g^2} + 1}$. The function $\frac{g}{\sqrt{g^2} + 1}$ is sigmoidal - it is like a smoothed step function. So, if $g$ is a very large positive or negative number, $g'_t$ will be 1 or -1 respectively. For smaller values of $g$, $g'_t$ gradually transitions from -1 to 1.